In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import torch

In [ ]:
# ۱. آماده‌سازی داده‌های آموزشی (متن‌های کوتاه برای یادگیری الگو)
texts = [
    "هوش مصنوعی آینده است",
    "هوش مصنوعی یادگیری ماشین دارد",
    "هوش مصنوعی قدرتمند است",
    "یادگیری ماشین بخشی از هوش مصنوعی است"
]
dataset = Dataset.from_dict({"text": texts})

In [ ]:
# ۲. آماده‌سازی توکنایزر
# نکته: GPT-2 به طور پیش‌فرض توکن Padding ندارد، پس باید آن را دستی تنظیم کنیم.
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=10)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
# ۳. تعریف معماری کاستوم Only-Decoder (GPT-2 کوچک)
config = AutoConfig.from_pretrained(
    "gpt2",
    n_layer=2,         # فقط ۲ لایه دکودر
    n_head=4,          # ۴ هد توجه
    n_embd=64,         # ابعاد بردارها
    vocab_size=len(tokenizer)
)

# ساخت مدل خام (بدون وزن‌های پیش‌آموزش‌دیده)
model = AutoModelForCausalLM.from_config(config)

In [ ]:
# ۴. استفاده از Data Collator مخصوص مدل‌های Causal
# این بخش به طور خودکار برچسب‌ها (Labels) را برای پیش‌بینی کلمه بعدی می‌سازد
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# ۵. تنظیمات و اجرای آموزش
training_args = TrainingArguments(
    output_dir="./gpt-mini-results",
    num_train_epochs=200,      # تعداد اپوک بالا چون داده‌ها بسیار کم هستند
    per_device_train_batch_size=2,
    logging_steps=50,
    learning_rate=1e-3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

print("شروع آموزش مدل مولد...")
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


شروع آموزش مدل مولد...


Step,Training Loss
50,7.030858
100,2.455615
150,0.728925
200,0.247740
250,0.107635
300,0.068170
350,0.052720
400,0.047333


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=400, training_loss=1.3423743611574173, metrics={'train_runtime': 21.7079, 'train_samples_per_second': 36.853, 'train_steps_per_second': 18.427, 'total_flos': 4804608000.0, 'train_loss': 1.3423743611574173, 'epoch': 200.0})

In [ ]:
# ۶. تست مدل: تولید متن (Inference)
# استفاده از متد قدرتمند generate
prompt = "هوش مصنوعی"
inputs = tokenizer(prompt, return_tensors="pt")

# مدل شروع به حدس زدن کلمات بعدی می‌کند
output_tokens = model.generate(
    **inputs,
    max_new_tokens=10,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(f"\nورودی: {prompt}")
print(f"خروجی تولید شده: {generated_text}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



ورودی: هوش مصنوعی
خروجی تولید شده: هوش مصنوعیادگعع��ر�
